In [4]:
using Revise

using Bilevel
using LinearAlgebra
using ForwardDiff

In [5]:
a = rand(4)
A = Matrix(Diagonal(a))

4×4 Array{Float64,2}:
 0.696328  0.0       0.0       0.0     
 0.0       0.466737  0.0       0.0     
 0.0       0.0       0.700535  0.0     
 0.0       0.0       0.0       0.812809

In [6]:
# test the solution
function f(x)
    return sum(x)
end

function h(x)
    return [6. .- x[1], x[1]*x[2], x[1]*.5 + x[3] - 4.]
end

function g(x)
    return vcat(-x[2:3],5. - x[4],10. - x[4],17. - x[4],5. - x[1]*x[4])
end

x0 = zeros(4)
λ0 = zeros(length(h(x0)))
μ0 = zeros(length(g(x0)))
c0 = 1.

x_sol, λ_sol, μ_sol, c_sol = auglag_solve(x0,λ0,μ0,f,h,g,c0=c0)

num_h = length(λ0)
num_g = length(μ0)
x_sol_ip = ip_solve(x0,f,h,g,num_h,num_g)

println(x_sol)
println(x_sol_ip)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Solve_Succeeded
[6.0, -3.62314e-14, 1.0, 17.0]
[6.0, 2.16408e-24, 1.0, 17.0]


In [7]:
# test the gradient
function solve_prob(z)
    function f(x)
        return sum(x)
    end

    function h(x)
        return [z[1] - x[1], x[1]*x[2], x[1]*.5 + x[3] - z[2]]
    end

    function g(x)
        return vcat(-x[2:3], z[3] - x[4], 10. - x[4], z[4] - x[4], 5. - x[1]*x[4])
    end

    x0 = zeros(4)
    λ0 = zeros(length(h(x0)))
    μ0 = zeros(length(g(x0)))

    # x_sol_ip = ip_solve(x0,f,h,g,length(h(x0)),length(g(x0)))
    # println(x_sol_ip)
    
    x_sol, λ_sol, μ_sol, c_sol = auglag_solve(x0,λ0,μ0,f,h,g)
    # println(x_sol)

    x_sol
end

solve_prob (generic function with 1 method)

In [8]:
# z0 = [2.,-1.,0.,1.]
z0 = [6.,4.,5.,17.]

sol = solve_prob(z0)

# autodiff 
J_auto = ForwardDiff.jacobian(solve_prob,z0)

# numerical
ϵ = sqrt(eps(1.))
J_num = zeros(size(J_auto))
for i = 1:length(z0)
    δ = zeros(length(z0))
    δ[i] = ϵ
    J_num[:,i] = (solve_prob(z0 + δ) - sol) ./ ϵ
end

println("----")
display(sol)
display(J_auto)
display(J_num)
display(maximum(abs.(J_auto - J_num)))

4-element Array{Float64,1}:
  5.999999999999564     
 -3.6231394263129694e-14
  1.0000000000052178    
 16.99999999999673      

4×4 Array{Float64,2}:
  1.0         -2.98088e-11   1.9948e-14   -4.24997e-8
  3.60948e-9   1.33249e-11  -8.25543e-13   1.30004e-9
 -0.5          1.0          -7.9305e-13    1.99581e-8
 -4.2722e-8    3.92592e-12   1.40582e-13   1.0       

4×4 Array{Float64,2}:
  0.999984     -1.07288e-5   2.2471e-5     4.71473e-5 
  9.33272e-6    3.00903e-5  -1.17446e-5   -1.78636e-5 
 -0.500391      0.999711    -0.000314832  -0.000259265
  0.000228167   0.0002141    0.000403643   1.00019    

0.00040364265427836327

----


In [97]:
display(maximum(abs.(J_auto - J_num)))

0.002547256759153016